In [2]:
import torch
import coremltools as ct

# Pytorch Grid Model
class PytorchGridSample(torch.nn.Module):
    def forward(self, input, grid):
        return torch.nn.functional.grid_sample(input, grid, align_corners=False)

def convert_to_coreml(model, inputs, is_float16=True):
    try:
        traced_model = torch.jit.trace(model, example_inputs=inputs, strict=False)

        # Precision
        precision = ct.precision.FLOAT16 if is_float16 else ct.precision.FLOAT32
        coreml_model = ct.converters.convert(
            traced_model,
            inputs=[ct.TensorType(shape=inputs[0].shape),
                    ct.TensorType(shape=inputs[1].shape)],
            compute_precision=precision
        )

        return coreml_model
    except Exception as e:
        print(f"Error during CoreML conversion: {e}")
        return None

def compare_grid_samples_after_coreml_conversion(pt_model, inputs, is_float16):
    try:
        pt_out = pt_model(*inputs)
        coreml_pt_model = convert_to_coreml(pt_model, inputs, is_float16)

        input_names_coreml_pt = [i for i in coreml_pt_model.input_description]
        input_data = {name: val.detach().numpy() for name, val in zip(input_names_coreml_pt, inputs)}

        coreml_pt_out = torch.as_tensor(list(coreml_pt_model.predict(input_data).values())[0])

        # Uncomment the line below to introduce an intentional issue in CoreML
        # coreml_pt_out[0, 0, 0, 0] += 1.0

        diff_pt_coreml = torch.norm(coreml_pt_out - pt_out)
        return diff_pt_coreml
    except Exception as e:
        print(f"Error during comparison: {e}")
        return None

if __name__ == "__main__":
    try:
        # Load input tensors from .pt files
        input_tensor = torch.load("feat_tensor.pt").to(torch.float32)
        grid = torch.load("grid_tensor.pt").to(torch.float32)
        inputs = [input_tensor, grid]

        pt_model = PytorchGridSample()

        diff_pt_coreml_fp16 = compare_grid_samples_after_coreml_conversion(pt_model, inputs, is_float16=True)
        diff_pt_coreml_fp32 = compare_grid_samples_after_coreml_conversion(pt_model, inputs, is_float16=False)

        if diff_pt_coreml_fp16 is not None and diff_pt_coreml_fp32 is not None:
            print(f"Difference between PyTorch's grid sample before and after conversion: Note: PyTorch is fp32 and CoreML is fp16: {diff_pt_coreml_fp16}")
            print(f"Difference between PyTorch's grid sample before and after conversion: Note: PyTorch is fp32 and CoreML is fp32: {diff_pt_coreml_fp32}")
            print(f"Relative change in the difference: {(diff_pt_coreml_fp16 - diff_pt_coreml_fp32) / diff_pt_coreml_fp32}")
    except Exception as e:
        print(f"Unexpected error: {e}")

When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_targer' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://coremltools.readme.io/docs/unified-conversion-api#target-conversion-formats
Model is not in eval mode. Consider calling '.eval()' on your model prior to conversion
Running MIL default pipeline:   0%|          | 0/71 [00:00<?, ? passes/s]/Users/surajmahapatra/Library/Python/3.9/lib/python/site-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:267: UserWarning: Output, '6', of the source model, has been renamed to 'var_6' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12

Difference between PyTorch's grid sample before and after conversion: Note: PyTorch is fp32 and CoreML is fp16: 5.701563350157812e-05
Difference between PyTorch's grid sample before and after conversion: Note: PyTorch is fp32 and CoreML is fp32: 5.701563350157812e-05
Relative change in the difference: 0.0
